# Unsupervised Learning

- $k$-means and other clustering approaches
- principal component analysis (PCA)
- probabilistic PCA and latent variable models

In [1]:
import pods
import notebook as nb
import mlai
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
%matplotlib inline

### Review and Preview

* last week we saw Bayesian assigns a probability distribution on parameters
* concept of conjugate priors was introduced
* we saw Bayesian regularises the means
* this week we look at unsupervised learning, in particular, $k$-means and PCA

### Supervised Learning

Supervised learning is learning where each data has a label.

- training example: set of pairs consisting of an input object and a desired output value
- recall the bias-variance tradeoff (**week 5**)

(eg) regression analysis

Wikipedia: [Supervised learning](https://en.wikipedia.org/wiki/Supervised_learning)

### Unsupervised Learning

In unsupervised learning we have no labels for the data.
It is often thought of as structure discovery, such as finding features in the data.

Unsupervised learning techniques:
- clustering (eg: k-means)
- latent variable models (eg: EM algorithm)
- blind signal separation (eg: PCA, ICA, SVD)

Wikipedia: [Unsupervised learning](https://en.wikipedia.org/wiki/Unsupervised_learning)

### Clustering

Clustering is a class of important and widely used unsupervised learning techniques (but it is not covered very much in this module).

It associate each data point $y_i$ with one of $k$ different discrete groups:
- clustering animals into discrete groups (Are animals discrete or continuous?)
- clustering into different different political affiliations

Wikipedia: [Cluster analysis](https://en.wikipedia.org/wiki/Cluster_analysis)

(next graph) clustering example of two dimensional dataset

In [2]:
nb.display_plots('cluster_data{counter:0>2}.svg', directory='./diagrams', counter=(0, 1))

### Vector Quantisation (VQ)

VQ determines how to allocate each point to a group.
- *harder* total number of groups
- conceptually similar to clustering

Wikipedia: [Vector quantization](https://en.wikipedia.org/wiki/Vector_quantization)

### $K$-means Clustering

$k$-means is a simple algorithm for allocating $n$ data points into $K$ groups. 
It requires a set of $K$ cluster centres and the algorithm assigns each points to a cluster:
1. initialize cluster centres as randomly selected data points
2. assign each data point to the nearest cluster centre
3. update each cluster centre by setting it to the mean of assigned data points
4. repeat 2 and 3 until cluster allocations do not change

Wikipedia: [K-means clustering](https://en.wikipedia.org/wiki/K-means_clustering)

### $K$-means Objective Function

$K$-means minimises the following objective function:
$$
  E = \sum_{j=1}^K \sum_{i\ \text{allocated to}\ j} \left( \mathbf{y}_i - \boldsymbol{\mu}_j \right)^\top \left(\mathbf{y}_i - \boldsymbol{\mu}_j \right)
$$
ie, it minimises the sum of Euclidean squared distances between data points $\mathbf{y}_i$ and their associated centres $\boldsymbol{\mu}_j$.

- this objective function is a non-convex optimisation problem
- the minimum is NOT guaranteed to be global or unique

(next graph) 4 iterations of $k$-means clustering

In [3]:
nb.display_plots('kmeans_clustering_{counter:0>3}.svg', directory='./diagrams', text_top='kmeans_clustering_{counter:0>3}.tex', counter=(0, 5))

### Other Clustering Approaches (I)

**Spectral clustering** :
- allowing clusters which aren't convex hulls

Wikipedia: [Spectral clustering](https://en.wikipedia.org/wiki/Spectral_clustering)

### Other Clustering Approaches (II)

**Dirichlet process** :
- a probabilistic formulation for a clustering algorithm that is non-parametric
- (loosely speaking) allowing infinite clusters
- in practice useful for dealing with previously unknown species (eg. a **black swan event**)

Wikipedia: [Dirichlet process](https://en.wikipedia.org/wiki/Dirichlet_process)

Wikipedia: [Black swan theory](https://en.wikipedia.org/wiki/Black_swan_theory)

### High Dimensional Data

USPS dataset for handwritten digits:
- 3648 dimensions (64 rows, 57 columns)

(next graph) space of 64 rows and 57 columns, including USPS sample '6'
- space contains many more than just '6'
- even if sampled at every nanonsecond from now until end of universe you won't see the original '6'

In [4]:
nb.display_plots('dem_six{counter:0>3}.png', directory='./diagrams', counter=(0, 3))

### Low Dimensional Manifolds

(observation)
Digit '6' may be rotated, but pure rotation is too simple and, in practice, data may undergo several distortions.

- for high dimensional data with structure, we expect fewer distortions than dimensions, ie, expect the data to live on a lower dimensional manifold

(conclusion)
High dimensional data may be dealt with by looking for a lower dimensional (non-)linear embedding.

- $m$-dimensional data $\mathbf{y}_i$ may be mapped to $d$-dimentional representation $\mathbf{x}_i$, where $m>d$

### Principal Component Analysis (PCA)

PCA is a linear embedding, which is presented as rotating the space to find **directions in data with maximal variance**.

How do we find these directions?

- diagonalising the sample covariance matrix $\mathbf{S}$ :
$$
  \mathbf{S} = \frac{1}{n} \sum_{i=1}^n \mathbf{y}_i \mathbf{y}_i^\top
$$
for a set of $n$ data points $\mathbf{y}_i$ with **zero mean**.

- if $\boldsymbol{\mu} \neq 0$, shift $\mathbf{y}_i$ by $\boldsymbol{\mu}$ before calculating the covariance.

Wikipedia: [Principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis)

### Before PCA

<img src="https://upload.wikimedia.org/wikipedia/commons/a/a6/PCA_linear_start.png", width=500>

- covariance has large values with non-diagonal elements

### After PCA

<img src="https://upload.wikimedia.org/wikipedia/commons/6/63/PCA_linear_end.png", width=500>

- covariance is diagonal, or having very small values with non-diagonal elements

### PCA: Task Definition

Here our view of an $m$-dimensional vector $\mathbf{w}_1$ is a line through the origin.
For simplicity we use $\mathbf{w}_1$ having a unit length.
The projection of a single, $m$-dimensional data point $\mathbf{y}_i$ onto a line $\mathbf{w}_1$ is given by
$$
  \frac{\mathbf{w}_1^\top}{||\mathbf{w}_1||} \mathbf{y}_i = \mathbf{w}_1^\top \mathbf{y}_i
$$
The covariance of the projected dataset is
$$
  \frac{1}{n} \sum_{i=1}^n (\mathbf{w}_1^\top \mathbf{y}_i) (\mathbf{w}_1^\top \mathbf{y}_i)^\top
  = \mathbf{w}_1^\top \left( \frac{1}{n} \sum_{i=1}^n \mathbf{y}_i \mathbf{y}_i^\top \right) \mathbf{w}_1
  = \mathbf{w}_1^\top \mathbf{S} \mathbf{w}_1
$$
Our task is to find directions for which the variance above is maximised.

### PCA: Lagrangian

The solution is found by constrained optimisation using a Lagrange multiplier $\lambda_1$ :
$$
  L(\mathbf{w}_1,\lambda_1) = \mathbf{w}_1^\top \mathbf{S} \mathbf{w}_1 + \lambda_1 (1 - \mathbf{w}_1^\top \mathbf{w}_1)
$$
The gradient with respect to $\mathbf{w}_1$ is
$$
  \nabla_{\mathbf{w}_1} L = \frac{\partial L(\mathbf{w}_1,\lambda_1)}{\partial \mathbf{w}_1} = 2\mathbf{S}\mathbf{w}_1 - 2\lambda_1 \mathbf{w}_1
$$
We set $\nabla_{\mathbf{w}_1} L = 0$ and rearrange to form
$$
  \mathbf{S} \mathbf{w}_1 = \lambda_1 \mathbf{w}_1
$$
which is known as an **eigenvalue problem**.

Wikipedia:
[Lagrange multiplier](https://en.wikipedia.org/wiki/Lagrange_multiplier) /
[Eigenvalues and eigenvectors](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors)

### PCA: Projection of a Single Data Point

- the optimised solution $\mathbf{S} \mathbf{w}_1 = \lambda \mathbf{w}_1$ indicates that the desired direction $\mathbf{w}_1$ is an eigenvector of $\mathbf{S}$

- further directions $\mathbf{w}_2,\mathbf{w}_3,\ldots$ that are **orthogonal** to $\mathbf{w}_1$ can also be shown to be eigenvectors of the covariance $\mathbf{S}$

Arranging eigenvectors to create an $m\times d$ matrix:
$$
  \mathbf{W}
  = (\mathbf{w}_1, \ldots, \mathbf{w}_d)
  = \left(\begin{array}{ccc} w_{11} & \cdots & w_{1d} \\ \vdots & & \vdots \\ w_{m1} & \cdots & w_{md} \end{array}\right)
$$
and $d$-dimensional projection $\mathbf{x}_i$ of an $m$-dimentional data point $\mathbf{y}_i$ is
$$
  \mathbf{x}_i = \mathbf{W}^\top \mathbf{y}_i
$$

### PCA: Projection of a Dataset

Projection $\mathbf{X}$ of a dataset $\mathbf{Y}$, consisting of $n$ data points, is given by
$$
\mathbf{X} = \mathbf{Y}\mathbf{W}
$$
- $m\times d$ matrix $\mathbf{W}$ of eigenvectors is as before
- $n\times d$ projection $\mathbf{X}$ and $n\times m$ dataset $\mathbf{Y}$ are given by
$$
  \mathbf{X}
  = \left(\begin{array}{c} \mathbf{x}_1^\top \\ \vdots \\ \mathbf{x}_n^\top \end{array}\right)
  = \left(\begin{array}{ccc} x_{11} & \cdots & x_{1d} \\ \vdots & & \vdots \\ x_{n1} & \cdots & x_{nd} \end{array}\right)
  \qquad
  \mathbf{Y}
  = \left(\begin{array}{c} \mathbf{y}_1^\top \\ \vdots \\ \mathbf{y}_n^\top \end{array}\right)
  = \left(\begin{array}{ccc} y_{11} & \cdots & y_{1m} \\ \vdots & & \vdots \\ y_{n1} & \cdots & y_{nm} \end{array}\right)
$$

### PCA:  Eigenvalue Decomposition

In practice eigenvectors can be derived by **eigenvalue decomposition** of a covariance.

- $n\times n$ covariance matrix $\mathbf{S}$ can be factorised as
$$
  \mathbf{S} = \mathbf{V} \boldsymbol{\Lambda} \mathbf{V}^{-1}
$$
where $\mathbf{V}$ is $n\times n$ matrix whose columns are the eigenvectors of $\mathbf{S}$ and $\boldsymbol{\Lambda}$ is the diagonal matrix whose diagonal elements are the corresponding eigenvalues

Alternative is to use **singular value decomposition** (**SVD**).

Wikipedia:
[Eigendecomposition of a matrix](https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix) /
[SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition)

### PCA: Recap

Let $\lambda_1, \lambda_2, \ldots$ be eigenvalues of $\mathbf{S}$ in non-increasing order, and $\mathbf{w}_1, \mathbf{w}_2, \ldots$ be corresponding eigenvectors:

- PCA changes the basis: projection basis is a **linear combination** of measurement basis
- $\mathbf{w}_1, \mathbf{w}_2, \ldots$ are principal components that are **orthogonal**
- $\mathbf{w}_1$ is the direction preserving the largest variance, and the resulting variance is simply $\lambda_1$
- $\mathbf{w}_i$ is the direction preserving the $i^{th}$ largest variance
- large variances pertain the important structure of a dataset

### Latent Variables

Limitation of PCA:
- no probabilistic model for observed data
- computation intensive for large dataset with high dimension
- missing data problem
- affected by outliers

In the following we introduce **latent variables** (hidden, or unobserved variables) to represent data and their dependencies in lower dimension.

Wikipedia: [Latent variable](https://en.wikipedia.org/wiki/Latent_variable)

### Latent Variable Model

There are $n$ data points: $\mathbf{y}_i$ that are observable in the $m$-dimensional space.
We would like to represent each $\mathbf{y}_i$ using a latent variable $\mathbf{x}_i$ in the $d$-dimensional space.
Note that $d<m$.

We assume that latent variables relate to observed data points through principal axes $\mathbf{W}$, which is an $m\times d$ matrix.
This leads to a linear relationship of the form:
$$
  \mathbf{y}_i = \mathbf{W} \mathbf{x}_i + \boldsymbol{\epsilon}_i
$$
where $\boldsymbol{\epsilon}_i \sim \mathcal{N}(\mathbf{0}, \sigma^2\mathbf{I})$ with a noise term $\sigma^2$ .

### Probabilistic PCA

We assume that each latent variable is normally distributed:
$$
  \mathbf{x}_i \sim \mathcal{N}(\mathbf{0},\mathbf{I})
$$

The corresponding data point $\mathbf{y}_i$ is generated by projection:
$$
  \mathbf{y}_i | \mathbf{x}_i \sim \mathcal{N}(\mathbf{W}\mathbf{x}_i,\sigma^2\mathbf{I})
$$

**Probabilistic PCA** aims to estimate the principal axes $\mathbf{W}$ and the noise $\sigma^2$ .

### Probabilistic PCA

Firstly, $\mathbf{x}_i \sim \mathcal{N}(\mathbf{0},\mathbf{I})$ implies that
$$
  \mathbf{W} \mathbf{x}_i \sim \mathcal{N}(\mathbf{0}, \mathbf{W}\mathbf{W}^\top)
$$

Further, given assumption $\boldsymbol{\epsilon}_i \sim \mathcal{N}(\mathbf{0}, \sigma^2\mathbf{I})$ with a linear relation $\mathbf{y}_i = \mathbf{W} \mathbf{x}_i + \boldsymbol{\epsilon}_i$,
$$
  \mathbf{y}_i \sim \mathcal{N}(\mathbf{0}, \mathbf{W}\mathbf{W}^\top + \sigma^2\mathbf{I})
$$

**Classical PCA** is the special case of **probabilistic PCA** when the covariance of the noise appraoches zero, ie, $\sigma^2 \rightarrow 0$ .

### Probabilistic PCA

The standard approach to solution using latent variable models is
1. difine a Gaussian prior over latent space $\mathbf{X}$
2. integrate out latent variables

In probabilistic PCA marginal likelihood for data matrix $\mathbf{Y}$ has the form:
$$
  p(\mathbf{Y} | \mathbf{W},\sigma^2) = \prod_{i=1}^n \mathcal{N}(\mathbf{y}_i | \mathbf{0}, \mathbf{W}\mathbf{W}^\top + \sigma^2\mathbf{I})
$$